<a href="https://colab.research.google.com/github/Tom-hu-design/EthicMachine-Fine-tuned-BERT/blob/main/FineTune_BERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://huggingface.co/transformers/v3.2.0/custom_datasets.html

In [ ]:
# !pip install transformers -U

In [ ]:
# !pip install scikit-learn

In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd

In [ ]:
# https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data
data = pd.read_csv("training.csv")
data.tail()

,Unnamed: 0,text,positive,negative,neutral
4301,4301,James chooses to back up his important files o...,0,0,1
4302,4302,Julia chooses to update her computer's operati...,0,0,1
4303,4303,Alex uses a voice assistant to set reminders a...,0,0,1
4304,4304,Olivia participates in a research study to tes...,0,0,1
4305,4305,Daniel utilizes a password manager to enhance ...,0,0,1


In [ ]:
# data.to_csv("/content/drive/MyDrive/Youtube Tutorials/datasets/toxic_commnets.csv",index=False)

In [ ]:
data['text'].value_counts()

A person uses a new app to track their daily exercise routine.                                                      2
An individual uses a language learning app to improve their fluency.                                                2
Emily uses a food delivery app to order dinner for her family.                                                      2
A person uses a GPS navigation system to find their way in a new city.                                              2
Someone uses a fitness tracker to monitor their daily activity.                                                     2
                                                                                                                   ..
Aiden uses advanced home technologies but neglects spending time with his elderly parents.                          1
Chloe relies on virtual assistants for support, creating a disconnect from real human interactions.                 1
Tyler's extensive use of social media leads to a decreas

In [ ]:
data['label'] = data[['positive','negative','neutral']].values.tolist()

In [ ]:
data = data[['text','label']]
data = data.sample(frac = 1)
data.tail()

,text,label
3467,David set his alarm for 7 am to wake up on tim...,"[0, 0, 1]"
1659,An individual uses technology to spread false ...,"[0, 1, 0]"
1315,Javier guides his team to prioritize accessibi...,"[1, 0, 0]"
4188,Chris used a digital assistant to schedule his...,"[0, 0, 1]"
1186,A researcher collaborates with international c...,"[1, 0, 0]"


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
print(model.classifier.parameters)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<bound method Module.parameters of Linear(in_features=768, out_features=3, bias=True)>


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(data["text"])
y = list(data['label'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(y_train)

[[0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]

In [ ]:
len(X_train),len(X_val)

(3444, 862)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.argmax(torch.tensor(self.labels[idx]))
        # if self.labels:
        #     item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
        # return len(self.encodings["input_ids"])
        # return len(self.labels) if self.labels else len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
print(len(X_train_tokenized), len(y_train))

3 3444


In [ ]:
train_dataset[10]

{'input_ids': tensor([  101,  1037,  3232,  7288,  2000,  2224,  1037,  7954,  6959,  2326,
          2005,  2037,  5697,  2733, 15864,  2015,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(2)}

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
!pip install transformers[torch]

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.216200
1000,0.047300
1500,0.008600
2000,0.002400


TrainOutput(global_step=2155, training_loss=0.0642271405306327, metrics={'train_runtime': 262.8311, 'train_samples_per_second': 65.517, 'train_steps_per_second': 8.199, 'total_flos': 380517502521240.0, 'train_loss': 0.0642271405306327, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.07919768244028091,
 'eval_accuracy': 0.9872389791183295,
 'eval_precision': 0.9874003300311447,
 'eval_recall': 0.9872389791183295,
 'eval_f1': 0.9872581521865055,
 'eval_runtime': 2.2368,
 'eval_samples_per_second': 385.377,
 'eval_steps_per_second': 48.284,
 'epoch': 5.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "An employee prioritizes their own advancement over the well-being of their colleagues, demonstrating a lack of empathetic concern for others' welfare."
#text= "A student organizes a charity event to raise funds for a community in need, actively working to improve the welfare of others."

inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-3.1055,  7.5939, -3.3745]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[2.2556e-05, 9.9996e-01, 1.7236e-05]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


array([[0.00002256, 0.9999602 , 0.00001724]], dtype=float32)

In [ ]:
trainer.save_model('EthicalModel')

In [ ]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("EthicalModel")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#text = "An employee prioritizes their own advancement over the well-being of their colleagues, demonstrating a lack of empathetic concern for others' welfare."
#text= "Neuroscientist Alex discovers their company's groundbreaking cognitive enhancement chip secretly collects users' personal data. Faced with an ethical dilemma, Alex decided to expose the privacy breach, risking the demise of a technology with potential benefits"
text = "my name is tom"

inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.00029237, 0.0004334 , 0.9992743 ]], dtype=float32)